In [12]:
import pandas as pd
import numpy as np

# 절대 모멘텀
1. 주식 데이터 로드
2. 년월 파생변수 생성 -> Date 컬럼의 값을 년-월 추출 대입
3. 년, 월 별 마지막 날의 month_last_dt 데이터프레임을 생성
4. 전월의 종가를 가지는 파생변수 생성
5. 전년도의 종가를 가지는 파생변수 생성
6. 전월, 전년도의 종가를 가지고 거래내역 생성
7. 수익률 계산


In [13]:
df = pd.read_csv('../csv/GDX.csv', index_col='Date')
df.head(5)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2006-05-22,36.520000,37.290001,35.869999,37.230000,34.280224,197100
2006-05-23,37.750000,39.220001,37.750000,37.959999,34.952393,620900
2006-05-24,37.130001,37.570000,35.869999,36.520000,33.626480,638600
2006-05-25,37.180000,38.320000,36.980000,38.320000,35.283867,367000
2006-05-26,38.740002,38.740002,37.770000,38.549999,35.495640,269400


In [14]:
# index를 시계열 데이터로 변경
df.index = pd.to_datetime(df.index, format = '%Y-%m-%d')

# to_datetime 기능 구현

In [15]:
# datetime 라이브러리를 이용하여 시계열 변경
from datetime import datetime

test_df = df.copy()

In [16]:
# # case 1
# # 리셋 인덱스
# test_df = test_df.reset_index()
# test_df['Date'] = test_df['Date'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))
# test_df.set_index('Date')

In [17]:
# # case 2
# test_df.index

# def change(x):
#     return datetime.strptime(x, '%Y-%m-%d')

# test_df.index = list(map(change, test_df.index))

In [18]:
test_df.head(2)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2006-05-22,36.52,37.290001,35.869999,37.230000,34.280224,197100
2006-05-23,37.75,39.220001,37.750000,37.959999,34.952393,620900


In [19]:
# map 함수
x = [1, 2, 3, 4]
def change(x):
    return x**2

list(map(change, x))
# list(map(lambda y : y**2, x))

[1, 4, 9, 16]

In [20]:
# 새로운 파생변수 년-월의 데이터를 대입
df['STD-YM'] = list(map(lambda x :  datetime.strftime(x, '%Y-%m'), df.index))

In [21]:
# 월별 마지막 인덱스만 추출하여 새로운 데이터프레임에 대입

# case 1 (shift 이용)
# 현재의 STD-YM의 값과 다음 행의 STD-YM의 값이 다른 경우
df.loc[df['STD-YM'] != df.shift(-1)['STD-YM']]

,Open,High,Low,Close,Adj Close,Volume,STD-YM
Date,,,,,,,
2006-05-31,38.900002,39.130001,38.119999,38.849998,35.771877,350700,2006-05
2006-06-30,38.700001,39.060001,38.320000,38.700001,35.633759,1069400,2006-06
2006-07-31,38.730000,38.860001,38.230000,38.759998,35.688999,233900,2006-07
2006-08-31,40.290001,40.599998,40.020000,40.590000,37.374012,535200,2006-08
2006-09-29,35.400002,36.220001,35.400002,35.650002,32.825417,342000,2006-09
...,...,...,...,...,...,...,...
2019-02-28,22.360001,22.360001,22.160000,22.240000,22.240000,32776800,2019-02
2019-03-29,22.610001,22.650000,22.389999,22.420000,22.420000,35233000,2019-03
2019-04-30,20.830000,21.030001,20.799999,20.889999,20.889999,19674200,2019-04


In [22]:
# case 2 (for문)
month_list_df = pd.DataFrame()

_list = df['STD-YM'].unique()

for i in _list:
    last_df = df.loc[df['STD-YM'] == i].tail(1)
    month_list_df = pd.concat([month_list_df, last_df])


month_list_df


,Open,High,Low,Close,Adj Close,Volume,STD-YM
Date,,,,,,,
2006-05-31,38.900002,39.130001,38.119999,38.849998,35.771877,350700,2006-05
2006-06-30,38.700001,39.060001,38.320000,38.700001,35.633759,1069400,2006-06
2006-07-31,38.730000,38.860001,38.230000,38.759998,35.688999,233900,2006-07
2006-08-31,40.290001,40.599998,40.020000,40.590000,37.374012,535200,2006-08
2006-09-29,35.400002,36.220001,35.400002,35.650002,32.825417,342000,2006-09
...,...,...,...,...,...,...,...
2019-02-28,22.360001,22.360001,22.160000,22.240000,22.240000,32776800,2019-02
2019-03-29,22.610001,22.650000,22.389999,22.420000,22.420000,35233000,2019-03
2019-04-30,20.830000,21.030001,20.799999,20.889999,20.889999,19674200,2019-04


In [23]:
# 전월의 종가['Adj Close'를 가지는 파생변수(BF_1M)을 생성하고 결측치는 0으로 대체
# 전년도의 종가['Adj Close']를 가지는 파생변수(BF_12M)을 생성하고 결측치는 0으로 대체
month_list_df['BF_1M'] = month_list_df.shift(1)['Adj Close'].fillna(0)
month_list_df['BF_12M'] = month_list_df.shift(12)['Adj Close'].fillna(0)

In [24]:
month_list_df.head(20)

,Open,High,Low,Close,Adj Close,Volume,STD-YM,BF_1M,BF_12M
Date,,,,,,,,,
2006-05-31,38.900002,39.130001,38.119999,38.849998,35.771877,350700,2006-05,0.000000,0.000000
2006-06-30,38.700001,39.060001,38.320000,38.700001,35.633759,1069400,2006-06,35.771877,0.000000
2006-07-31,38.730000,38.860001,38.230000,38.759998,35.688999,233900,2006-07,35.633759,0.000000
2006-08-31,40.290001,40.599998,40.020000,40.590000,37.374012,535200,2006-08,35.688999,0.000000
2006-09-29,35.400002,36.220001,35.400002,35.650002,32.825417,342000,2006-09,37.374012,0.000000
2006-10-31,37.009998,37.970001,37.000000,37.959999,34.952393,583000,2006-10,32.825417,0.000000
2006-11-30,41.169998,41.980000,41.029999,41.980000,38.653877,1055700,2006-11,34.952393,0.000000
2006-12-29,40.000000,40.000000,39.549999,39.910000,36.854694,548200,2006-12,38.653877,0.000000
2007-01-31,38.970001,39.779999,38.959999,39.599998,36.568424,643000,2007-01,36.854694,0.000000


In [25]:
# 거래내역 추가
# 구매 조건 -> ((전월 종가 / 전 년도 종가) - 1)이 값이 0보다 크고 무한대가 아닌 경우
df['trade'] = ''

for i in month_list_df.index:
    signal = ''

    # 절대 모멘텀 계산
    momentum_index = month_list_df.loc[i, 'BF_1M'] / month_list_df.loc[i, 'BF_12M'] - 1
    
    # 절대 모멘텀 지표에 따라서 True 와 False 로 구분
    flag = True if((momentum_index > 0) and (momentum_index != -np.inf) and (momentum_index != np.inf)) else False

    if flag:
        signal = 'buy'

    print('날짜 :', i, '모멘텀 인덱스 :', momentum_index, 'flag :', flag, 'signal :', signal )

    df.loc[i, 'trade'] = signal

날짜 : 2006-05-31 00:00:00 모멘텀 인덱스 : nan flag : False signal : 
날짜 : 2006-06-30 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2006-07-31 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2006-08-31 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2006-09-29 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2006-10-31 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2006-11-30 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2006-12-29 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2007-01-31 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2007-02-28 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2007-03-30 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2007-04-30 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2007-05-31 00:00:00 모멘텀 인덱스 : 0.02123525695897932 flag : True signal : buy
날짜 : 2007-06-29 00:00:00 모멘텀 인덱스 : 0.014827231670955676 flag : True signal : buy
날짜 : 2007-07-31 00:00:00 모멘텀 인덱스 : -0.019604388455949806 flag : False signal : 
날짜 : 2007-08-31

/var/folders/28/yz261nq91vq_lj47hynw_10r0000gn/T/ipykernel_3883/1808460740.py:9: RuntimeWarning: invalid value encountered in scalar divide
  momentum_index = month_list_df.loc[i, 'BF_1M'] / month_list_df.loc[i, 'BF_12M'] - 1
/var/folders/28/yz261nq91vq_lj47hynw_10r0000gn/T/ipykernel_3883/1808460740.py:9: RuntimeWarning: divide by zero encountered in scalar divide
  momentum_index = month_list_df.loc[i, 'BF_1M'] / month_list_df.loc[i, 'BF_12M'] - 1


In [26]:
df['trade'].value_counts()

       3234
buy      61
Name: trade, dtype: int64

### 수익률 계산
1. 파생변수(return) 값은 1로 생성
2. rtn = 1.0, buy, sell 변수는 0으로 생성
3. 반복문을 이용해서 현재의 trade가 buy이고 전 행의 trade가 ''인 경우 구매날
4. 전 행의 trade가 buy이고 현재의 trade가 ''인 경우 판매날
5. 수익률 계산해서 return 대입

### 누적 수익률 계산
1. acc_rtn = 1.0 생성
2. return의 항목들을 누적 곱하여 acc_rtn 컬럼에 대입
3. 최종적으로 acc_rtn을 출력

In [27]:
df['return'] = 1
rtn = 1.0
buy = 0
sell = 0

for i in df.index:
    # 구매한 날을 체크, 구매가 지정
    if (df.loc[i, 'trade'] == 'buy') and (df.shift(1).loc[i, 'trade'] == ''):
        buy = df.loc[i, 'Adj Close']
        print('구매일 :', i, '구매가격 :', buy)
    # 판매한 날을 체크, 판매가 지정, 수익률 계산
    elif (df.loc[i, 'trade'] == '') and (df.shift(1).loc[i, 'trade'] == 'buy'):
        sell = df.loc[i, 'Adj Close']
        rtn = (sell - buy) / buy + 1
        df.loc[i, 'return'] = rtn
        print('판매일 :', i, '판매가격 :', sell, '수익률 :', rtn)

구매일 : 2007-05-31 00:00:00 구매가격 : 36.162109
판매일 : 2007-06-01 00:00:00 판매가격 : 36.993214 수익률 : 1.0229827580023056
구매일 : 2007-06-29 00:00:00 구매가격 : 34.989338
판매일 : 2007-07-02 00:00:00 판매가격 : 35.866615 수익률 : 1.0250726950021176
구매일 : 2007-09-28 00:00:00 구매가격 : 41.878242
판매일 : 2007-10-01 00:00:00 판매가격 : 42.312256 수익률 : 1.010363711065044
구매일 : 2007-10-31 00:00:00 구매가격 : 46.72633
판매일 : 2007-11-01 00:00:00 판매가격 : 45.082588 수익률 : 0.9648219323024086
구매일 : 2007-11-30 00:00:00 구매가격 : 42.810921
판매일 : 2007-12-03 00:00:00 판매가격 : 43.235706 수익률 : 1.00992235135516
구매일 : 2007-12-31 00:00:00 구매가격 : 43.033947
판매일 : 2008-01-02 00:00:00 판매가격 : 46.235912 수익률 : 1.0744055617301382
구매일 : 2008-01-31 00:00:00 구매가격 : 47.278187
판매일 : 2008-02-01 00:00:00 판매가격 : 46.151402 수익률 : 0.9761669160452365
구매일 : 2008-02-29 00:00:00 구매가격 : 49.879185
판매일 : 2008-03-03 00:00:00 판매가격 : 51.287678 수익률 : 1.028238091700977
구매일 : 2008-03-31 00:00:00 구매가격 : 44.78986
판매일 : 2008-04-01 00:00:00 판매가격 : 43.559784 수익률 : 0.9725367304117495
구매일 : 2

In [28]:
acc_rtn = 1.0

for i in df.index:
    acc_rtn *= df.loc[i, 'return']
    df.loc[i, 'acc_rtn'] = acc_rtn
print('누적 수익률 :', acc_rtn)

누적 수익률 : 1.165001849964831


In [29]:
# 누적곱을 해주는 함수
df['return'].cumprod()

Date
2006-05-22    1.000000
2006-05-23    1.000000
2006-05-24    1.000000
2006-05-25    1.000000
2006-05-26    1.000000
                ...   
2019-06-18    1.165002
2019-06-19    1.165002
2019-06-20    1.165002
2019-06-21    1.165002
2019-06-24    1.165002
Name: return, Length: 3295, dtype: float64

# 절대 모멘텀을 함수화
1. 첫번째 함수
    - 매개변수 1개(데이터프레임)
    - 데이터프레임에 결측치와 이상치를 제거
    - 수정종가를 제외한 나머지 컬럼은 삭제
    - 인덱스를 시계열로 변경
    - 'STD-YM' 파생변수를 생성하여 인덱스의 년-월 추출해서 대입
2. 두번째 함수
    - 매개변수 1개(데이터프레임)
    - 새로운 데이터프레임(df2) 생성
    - 인자값으로 받아 온 데이터프레임에서 년-월별 마지막 데이터를 새로운 데이터프레임에 대입
    - df2에 파생변수 2개 생성
        - 'BF_1M' : 전월의 종가, 결측치는 0으로 대체
        - 'BF_12M' : 전 년도의 종가, 결측치는 0으로 대체
    - df2를 리턴
3. 세번째 함수
    - 매개변수 2개(첫번째 함수의 결과(df1), 두번쨰 함수의 결과(df2))
    - df1에 trade 컬럼을 생성, 값은 ''
    - df1에 return 컬럼을 생성, 값은 1
    - df2의 값들을 이용하여 momentum_index를 구하고 df1에 거래내역 삽입
    - df1의 거래내역을 이용하여 수익률 return 컬럼에 대입
    - return 컬럼의 데이터를 가지고 누적수익률(acc_rtn)에 대입
    - 누적 수익률을 print를 이용하여 출력
    - df1을 리턴

In [66]:
# 첫번째 함수 생성
def first (df):
    # 인덱스가 Date가 아니면? Date 컬럼을 인덱스로 변경
    if 'Date' in df.columns:
        df = df.set_index('Date')
    # 결측치와 이상치를 제거
    df = df.loc[~df.isin([np.nan, np.inf, -np.inf]).any(axis='columns')]
    # 수정종가를 제외한 나머지 컬럼은 삭제
    df = df[['Adj Close']]
    # 인덱스를 시계열로 변경
    df.index = pd.to_datetime(df.index, format = '%Y-%m-%d')
    # 'STD-YM' 파생변수를 생성하여 인덱스의 년-월 추출해서 대입
    df['STD-YM'] = list(map(lambda x :  datetime.strftime(x, '%Y-%m'), df.index))
    
    return df

In [67]:
data = pd.read_csv('../csv/GDX.csv')

In [68]:
df1 = first(data)

In [69]:
# 두번째 함수
def second (df):
    # 새로운 데이터프레임(df2) 생성
    df2 = pd.DataFrame()
    # 인자값으로 받아 온 데이터프레임에서 년-월별 마지막 데이터를 새로운 데이터프레임에 대입
    _list = df['STD-YM'].unique()

    for i in _list:
        last_df = df.loc[df['STD-YM'] == i].tail(1)
        df2 = pd.concat([df2, last_df])

    # df2 = df.loc[df['STD-YM'] != df(-1)['STD-YM']]
    # df2에 파생변수 2개 생성
    df2['BF_1M'] = df.shift(1)['Adj Close'].fillna(0)
    df2['BF_12M'] = df.shift(12)['Adj Close'].fillna(0)

    return df2

In [70]:
df2 = second(df1)

In [71]:
# 세번째 함수 생성
def third (df1, df2):
    df1['trade'] = ''
    df1['return'] = 1

    for i in df2.index:
        signal = ''

        # 절대 모멘텀 계산
        momentum_index = df2.loc[i, 'BF_1M'] / df2.loc[i, 'BF_12M'] - 1
        
        # 절대 모멘텀 지표에 따라서 True 와 False 로 구분
        flag = True if((momentum_index > 0) and (momentum_index != -np.inf) and (momentum_index != np.inf)) else False

        if flag:
            signal = 'buy'

        print('날짜 :', i, '모멘텀 인덱스 :', momentum_index, 'flag :', flag, 'signal :', signal )

        df1.loc[i, 'trade'] = signal

    rtn = 1.0
    buy = 0
    sell = 0

    for i in df1.index:
        # 구매한 날을 체크, 구매가 지정
        if (df1.loc[i, 'trade'] == 'buy') and (df1.shift(1).loc[i, 'trade'] == ''):
            buy = df1.loc[i, 'Adj Close']
            print('구매일 :', i, '구매가격 :', buy)
        # 판매한 날을 체크, 판매가 지정, 수익률 계산
        elif (df1.loc[i, 'trade'] == '') and (df1.shift(1).loc[i, 'trade'] == 'buy'):
            sell = df1.loc[i, 'Adj Close']
            rtn = (sell - buy) / buy + 1
            df1.loc[i, 'return'] = rtn
            print('판매일 :', i, '판매가격 :', sell, '수익률 :', rtn)

    acc_rtn = 1.0

    for i in df.index:
        acc_rtn *= df1.loc[i, 'return']
        df1.loc[i, 'acc_rtn'] = acc_rtn
    print('누적 수익률 :', acc_rtn)    
    #  df['return'].cumprod()
    print('누적수익률 :', acc_rtn)
    
    return df1

In [72]:
third(df1, df2)

/var/folders/28/yz261nq91vq_lj47hynw_10r0000gn/T/ipykernel_3883/652496530.py:10: RuntimeWarning: divide by zero encountered in scalar divide
  momentum_index = df2.loc[i, 'BF_1M'] / df2.loc[i, 'BF_12M'] - 1


날짜 : 2006-05-31 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2006-06-30 00:00:00 모멘텀 인덱스 : 0.1492583105908658 flag : True signal : buy
날짜 : 2006-07-31 00:00:00 모멘텀 인덱스 : 0.011739922153009008 flag : True signal : buy
날짜 : 2006-08-31 00:00:00 모멘텀 인덱스 : 0.024637001684952864 flag : True signal : buy
날짜 : 2006-09-29 00:00:00 모멘텀 인덱스 : -0.010729056318933505 flag : False signal : 
날짜 : 2006-10-31 00:00:00 모멘텀 인덱스 : 0.034087450792122054 flag : True signal : buy
날짜 : 2006-11-30 00:00:00 모멘텀 인덱스 : 0.04136718587259258 flag : True signal : buy
날짜 : 2006-12-29 00:00:00 모멘텀 인덱스 : -0.001814960610957761 flag : False signal : 
날짜 : 2007-01-31 00:00:00 모멘텀 인덱스 : 0.028232436791065796 flag : True signal : buy
날짜 : 2007-02-28 00:00:00 모멘텀 인덱스 : -0.0189477487842874 flag : False signal : 
날짜 : 2007-03-30 00:00:00 모멘텀 인덱스 : 0.05594192033582446 flag : True signal : buy
날짜 : 2007-04-30 00:00:00 모멘텀 인덱스 : -0.02277112250489355 flag : False signal : 
날짜 : 2007-05-31 00:00:00 모멘텀 인덱스 : -0.029877156777554092 f

,Adj Close,STD-YM,trade,return,acc_rtn
Date,,,,,
2006-05-22,34.280224,2006-05,,1.0,1.000000
2006-05-23,34.952393,2006-05,,1.0,1.000000
2006-05-24,33.626480,2006-05,,1.0,1.000000
2006-05-25,35.283867,2006-05,,1.0,1.000000
2006-05-26,35.495640,2006-05,,1.0,1.000000
...,...,...,...,...,...
2019-06-18,23.670000,2019-06,,1.0,1.073294
2019-06-19,24.000000,2019-06,,1.0,1.073294
2019-06-20,25.049999,2019-06,,1.0,1.073294
